In [2]:
!pip install --upgrade tensorflow
!pip install --upgrade tensorflow-io

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 42.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 76.8 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-io-gcs-filesystem
    Found existing installation: tensorflow-io-gcs-filesystem 0.31.0
    Uninstalling tensorflow-io-gcs-filesystem-0.31.0:
      Successfully uninstalled tensorflow-io-gcs-filesystem-0.31.0
  Attempting uninstall: tensorflow-io
    Found existing installation: tensorflow-io 0.31.0
    Uninstalling tensorflow-io-0.31.0:
      Successfully uninstalled tensorflow-io-0.31.0


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy
import re

In [4]:
data = pd.read_csv('/kaggle/input/biorxiv-cleancsv/biorxiv_clean.csv')

In [6]:
df = data
df = df.abstract.dropna()
dfabstracts = df.values.tolist()

len(dfabstracts)

776

# Summarization using T5 model

In [8]:
import torch
import os
import json

from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

In [10]:
T5_PATH = 'Einmalumdiewelt/T5-Base_GNAD'
t5_model = T5ForConditionalGeneration.from_pretrained(T5_PATH, output_past=True)
t5_tokenizer = T5Tokenizer.from_pretrained(T5_PATH)
x = t5_tokenizer(dfabstracts, truncation=True, max_length=500)

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def t5_summarize(input_text, num_beams=4, num_words=80):
    input_text = ' '.join(input_text.split())
    input_tokenized = t5_tokenizer.encode(input_text, return_tensors="pt").to(device)
    summary_task = torch.tensor([[21603, 10]]).to(device)
    input_tokenized = torch.cat([summary_task, input_tokenized], dim=-1).to(device)

    # Move the model to the same device as input tensors
    t5_model.to(device)

    summary_ids = t5_model.generate(input_tokenized,
                                    num_beams=int(num_beams),
                                    no_repeat_ngram_size=3,
                                    length_penalty=2.0,
                                    min_length=30,
                                    max_length=int(num_words),
                                    early_stopping=True)

    output = [t5_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) 
              for g in summary_ids]
    return output[0]


In [29]:
%%time
for i in range(6):
    
        print('BioArvix paper  ',i + 1, " : \n" )
        print(t5_summarize(dfabstracts[i]))
        print('............................................................................\n\n\n\n')


BioArvix paper   1  : 

61 62 63 64 65 66 69 79 81 82 83 84 95 92 93 94 95 96 97 NSP5 phosphorylation during RV infection. In this paper, we show that NSP5, which is required for the assembly of round-shaped viroplasms, plays a pivotal role in the
............................................................................




BioArvix paper   2  : 

In December 2019, an outbreak of novel coronavirus disease emerged in Wuhan, China and has rapidly spread to other parts of the country and oversea countries. We aim to evaluate the effectiveness of interventions and self-protection measures implemented by the Chinese government in mainland China excluding Hubei province.
............................................................................




BioArvix paper   3  : 

China is running a national level antivirus campaign against the novel coronavirus (2019-nCoV). The virus is closed to be under control, but economic loss has been caused by the virus. We recommend to adopt a proportio

In [14]:
def summarize_text(text):
    # Generate summary using the T5 model
    summary = t5_summarize(text)
    
    # Show the generated summary
    print("\nSummary:")
    print(summary)

# Prompt the user for input and generate summary
user_input = input("Enter the text to summarize: ")
summarize_text(user_input)


Enter the text to summarize:  There are numerous types of data structures, generally built upon simpler primitive data types. Well known examples are:[10]  An array is a number of elements in a specific order, typically all of the same type (depending on the language, individual elements may either all be forced to be the same type, or may be of almost any type). Elements are accessed using an integer index to specify which element is required. Typical implementations allocate contiguous memory words for the elements of arrays (but this is not always a necessity). Arrays may be fixed-length or resizable. A linked list (also just called list) is a linear collection of data elements of any type, called nodes, where each node has itself a value, and points to the next node in the linked list. The principal advantage of a linked list over an array is that values can always be efficiently inserted and removed without relocating the rest of the list. Certain other operations, such as random 


Summary:
Arrays, lists, tables, tables and tables are all examples of data structures. Object-oriented programming is based on primitive data structures, such as tables, graphs, trees and tries.


# Summarization using bart model

In [16]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

In [22]:
bart_PATH = 'facebook/bart-large-cnn'
bart_model = BartForConditionalGeneration.from_pretrained(bart_PATH, output_past=True)
bart_tokenizer = BartTokenizer.from_pretrained(bart_PATH)
x = bart_tokenizer(dfabstracts, truncation=True, max_length=500)

In [25]:
def bart_summarize(input_text, num_beams=4, num_words=80):
    #input_text = str(input_text)
    input_text = ' '.join(input_text.split())
    input_tokenized = bart_tokenizer.encode(input_text, return_tensors='pt')
    summary_ids = bart_model.generate(input_tokenized,
                                      num_beams=int(num_beams),
                                      no_repeat_ngram_size=3,
                                      length_penalty=2.0,
                                      min_length=80,
                                      max_length=int(num_words),
                                      early_stopping=True)
    output = [bart_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    return output[0]

In [27]:
%%time
for i in range(6):
    
        print('BioArvix paper  ',i + 1, " : \n" )
        print(bart_summarize(dfabstracts[i]))
        print('............................................................................\n\n\n\n')

BioArvix paper   1  : 

Rotavirus (RV) replicates in round-shaped cytoplasmic viral factories. During RV infection, NSP5 undergoes hyperphosphorylation, which is primed by the phosphorylation of a single serine residue. The role of this post-translational modification in the formation of viroplasms and its impact on the virus replication remains
............................................................................




BioArvix paper   2  : 

In December 2019, an outbreak of novel coronavirus disease emerged in Wuhan, China and has swiftly spread to other parts of China and a number of oversea countries. A novel COVID-19 transmission dynamic model incorporating the intervention measures implemented in China is proposed. If all the above conditions are met, the outbreak is expected to be ended by April in mainland China apart from
............................................................................




BioArvix paper   3  : 

China is running a national level antivirus cam

In [30]:
def summarize_text(text):
    # Generate summary using the bart model
    summary = bart_summarize(text)
    
    # Show the generated summary
    print("\nSummary:")
    print(summary)

# Prompt the user for input and generate summary
user_input = input("Enter the text to summarize: ")
summarize_text(user_input)

Enter the text to summarize:  There are numerous types of data structures, generally built upon simpler primitive data types. Well known examples are:[10]  An array is a number of elements in a specific order, typically all of the same type (depending on the language, individual elements may either all be forced to be the same type, or may be of almost any type). Elements are accessed using an integer index to specify which element is required. Typical implementations allocate contiguous memory words for the elements of arrays (but this is not always a necessity). Arrays may be fixed-length or resizable. A linked list (also just called list) is a linear collection of data elements of any type, called nodes, where each node has itself a value, and points to the next node in the linked list. The principal advantage of a linked list over an array is that values can always be efficiently inserted and removed without relocating the rest of the list. Certain other operations, such as random 


Summary:
An array is a number of elements in a specific order, typically all of the same type. Arrays may be fixed-length or resizable. A linked list is a linear collection of data elements of any type. A record is a value that contains other values, typically in fixed number and sequence. In the context of object-oriented programming, records are known as plain old data


# **evaluation metric** #

In [31]:
!pip install rouge-score
!pip install rouge

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=4a656bcd5ffd990d243f9d05eeeb35172feca2fd26d45084632e2b90792379a7
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [36]:
from rouge import Rouge

# Define the generated summary and reference summary
generated_summary_T5 = '''Arrays, lists, tables, tables and tables are all examples of data structures. Object-oriented programming is based on primitive data structures, such as tables, graphs, trees and tries.'''
reference_summary = '''Data structures are built upon primitive data types and include arrays, which store elements in a specific order and allow access through an index, and linked lists, which efficiently allow insertion and removal of values by pointing to the next node. Another type is records, which contain other values in a fixed sequence. Additional data structures include hash tables, graphs, stacks, queues, trees, binary trees, and tries.'''

# Initialize Rouge scorer
rouge_scorer = Rouge()

# Calculate ROUGE scores
rouge_scores = rouge_scorer.get_scores(generated_summary_T5,
                                       reference_summary)


print("ROUGE Scores:")
print(rouge_scores)


ROUGE Scores:
[{'rouge-1': {'r': 0.20754716981132076, 'p': 0.4782608695652174, 'f': 0.2894736799896122}, 'rouge-2': {'r': 0.06153846153846154, 'p': 0.15384615384615385, 'f': 0.08791208383045546}, 'rouge-l': {'r': 0.16981132075471697, 'p': 0.391304347826087, 'f': 0.23684210104224382}}]


In [35]:
from rouge import Rouge

# Define the generated summary and reference summary

generated_summary_bart = '''An array is a number of elements in a specific order, typically all of the same type. Arrays may be fixed-length or resizable. A linked list is a linear collection of data elements of any type. A record is a value that contains other values, typically in fixed number and sequence. In the context of object-oriented programming, records are known as plain old data'''
reference_summary = '''Data structures are built upon primitive data types and include arrays, which store elements in a specific order and allow access through an index, and linked lists, which efficiently allow insertion and removal of values by pointing to the next node. Another type is records, which contain other values in a fixed sequence. Additional data structures include hash tables, graphs, stacks, queues, trees, binary trees, and tries.'''

# Initialize Rouge scorer
rouge_scorer = Rouge()

# Calculate ROUGE scores
rouge_scores = rouge_scorer.get_scores(generated_summary_bart,
                                       reference_summary)


print("ROUGE Scores:")
print(rouge_scores)

ROUGE Scores:
[{'rouge-1': {'r': 0.2830188679245283, 'p': 0.3191489361702128, 'f': 0.29999999501800007}, 'rouge-2': {'r': 0.046153846153846156, 'p': 0.04918032786885246, 'f': 0.0476190426240872}, 'rouge-l': {'r': 0.24528301886792453, 'p': 0.2765957446808511, 'f': 0.25999999501800003}}]
